In [3]:
include("../load.jl")

  Activating new environment at `F:\Libraries\Documents\MIT\Research\Berk\OptimalConstraintTree\test\demos\test\Project.toml`


LoadError: LoadError: LoadError: ArgumentError: Package CPLEX not found in current path:
- Run `import Pkg; Pkg.add("CPLEX")` to install the CPLEX package.

in expression starting at F:\Libraries\Documents\MIT\Research\Berk\OptimalConstraintTree\src\GoML.jl:1
in expression starting at F:\Libraries\Documents\MIT\Research\Berk\OptimalConstraintTree\test\load.jl:24

In [ ]:
include("../../data/speed_reducer.jl")

In [ ]:
gm = speed_reducer()
uniform_sample_and_eval!(gm)
set_param(gm, :ignore_accuracy, true)

In [ ]:
feasibility_sample(gm)

In [ ]:
learn_constraint!(gm, max_depth=5)
add_tree_constraints!(gm)
optimize!(gm)
solution(gm)

In [ ]:
add_infeasibility_cuts!(gm)
optimize!(gm)
while abs(gm.cost[end] - gm.cost[end-1]) > 1e-10
    add_infeasibility_cuts!(gm)
    optimize!(gm)
end

In [ ]:
# Showing the iterations
show(stdout, MIME("text/latex"), gm.solution_history)
IAI.score(gm.bbls[1].learners[end], gm.bbls[1].X, gm.bbls[1].Y)

In [ ]:
# Checking that the solution is feasible
x = solution(gm)
for bbl in gm.bbls
    println(bbl.feas_gap[end])
end
IAI.show_in_browser(gm.bbls[6].learners[end])
# gm.bbls[6].constraint

In [ ]:
m = speed_reducer()
uniform_sample_and_eval!(m)
unset_integer(m.model[:x][3])
using Ipopt
set_optimizer(m, Ipopt.Optimizer)
set_optimizer_attribute(m.model, "constr_viol_tol", 1e-7)
nonlinearize!(m)
optimize!(m)

In [ ]:
x = solution(m)

In [ ]:
for bbl in m.bbls
    # @assert bbl(x)[1] == bbl.feas_gap[end] >= 0
    println(bbl.feas_gap[end])
end
(1 - m.bbls[1].feas_gap[end]) * m.bbls[1].actuals[end]

In [ ]:
m[x]